In [1]:
import os
import sys
import time
import numpy as np
import scipy.sparse as sp
from keras.layers import Input, Dropout
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

Using TensorFlow backend.


In [2]:
# the path of execution
EXE_PATH = os.path.abspath(os.path.curdir)
# the path of the vendor files
VENDOR_PATH = os.path.join(EXE_PATH, 'vendor')
# the vendors to include in the system path
VENDORS = ['keras-gcn']
# create the absolute paths for all vendors
VENDORS = list(map(lambda x: os.path.join(VENDOR_PATH, x), VENDORS))
# update the Python path to include necessary vendor module
sys.path += VENDORS

In [3]:
from kegra.layers.graph import GraphConvolution
from kegra.utils import load_data, preprocess_adj, get_splits, evaluate_preds
from ggcn import GaussianGraphConvolution, kl_reg

# Data

In [4]:
X, A, y = load_data('data/cora/', dataset='cora')
X /= X.sum(1).reshape(-1, 1)
A = preprocess_adj(A)
y_train, y_val, y_test, idx_train, idx_val, idx_test, train_mask = get_splits(y)

Loading cora dataset...
Dataset has 2708 nodes, 5429 edges, 1433 features.


In [5]:
X

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [6]:
A

<2708x2708 sparse matrix of type '<class 'numpy.float64'>'
	with 13264 stored elements in Compressed Sparse Row format>

In [7]:
y

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)

In [8]:
support = 1
graph = [X, A]
G = [Input(shape=(None, None), batch_shape=(None, None), sparse=True)]

W1110 18:28:15.185354 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1110 18:28:15.192788 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:515: The name tf.sparse_placeholder is deprecated. Please use tf.compat.v1.sparse_placeholder instead.



In [9]:
X_in = Input(shape=(X.shape[1],))

W1110 18:28:15.197443 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



# GCN (baseline)

In [10]:
# Define model architecture
# NOTE: We pass arguments for graph convolutional layers as a list of tensors.
# This is somewhat hacky, more elegant options would require rewriting the Layer base class.
H = Dropout(0.5)(X_in)
H = GraphConvolution(32, support, activation='relu', kernel_regularizer=l2(5e-4))([H]+G)
H = Dropout(0.5)(H)
Y = GraphConvolution(y.shape[1], support, activation='softmax')([H]+G)

W1110 18:28:15.204402 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W1110 18:28:15.209995 139734109144896 deprecation.py:506] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3445: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
W1110 18:28:15.218433 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))

W1110 18:28:15.259835 139734109144896 deprecation_wrapper.py:119] From /home/bitcommander/.local/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [12]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
graph_convolution_1 (GraphConvo (None, 32)           45888       dropout_1[0][0]                  
                                                                 input_1[0][0]                    
__________

In [13]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

W1110 18:28:15.336023 139734109144896 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch: 0001 train_loss= 1.9301 train_acc= 0.2929 val_loss= 1.9316 val_acc= 0.3500 time= 1.0769
Epoch: 0002 train_loss= 1.9141 train_acc= 0.2929 val_loss= 1.9172 val_acc= 0.3500 time= 0.0226
Epoch: 0003 train_loss= 1.8963 train_acc= 0.2929 val_loss= 1.9009 val_acc= 0.3500 time= 0.0229
Epoch: 0004 train_loss= 1.8770 train_acc= 0.2929 val_loss= 1.8836 val_acc= 0.3500 time= 0.0178
Epoch: 0005 train_loss= 1.8571 train_acc= 0.2929 val_loss= 1.8657 val_acc= 0.3500 time= 0.0250
Epoch: 0006 train_loss= 1.8373 train_acc= 0.2929 val_loss= 1.8481 val_acc= 0.3500 time= 0.0239
Epoch: 0007 train_loss= 1.8168 train_acc= 0.2929 val_loss= 1.8295 val_acc= 0.3500 time= 0.0227
Epoch: 0008 train_loss= 1.7969 train_acc= 0.2929 val_loss= 1.8117 val_acc= 0.3500 time= 0.0246
Epoch: 0009 train_loss= 1.7779 train_acc= 0.2929 val_loss= 1.7954 val_acc= 0.3500 time= 0.0191
Epoch: 0010 train_loss= 1.7597 train_acc= 0.2929 val_loss= 1.7807 val_acc= 0.3500 time= 0.0294
Epoch: 0011 train_loss= 1.7425 train_acc= 0.2929 v

Epoch: 0088 train_loss= 0.6978 train_acc= 0.9214 val_loss= 1.0075 val_acc= 0.7867 time= 0.0217
Epoch: 0089 train_loss= 0.6906 train_acc= 0.9143 val_loss= 0.9990 val_acc= 0.7900 time= 0.0195
Epoch: 0090 train_loss= 0.6850 train_acc= 0.9000 val_loss= 0.9919 val_acc= 0.7933 time= 0.0224
Epoch: 0091 train_loss= 0.6786 train_acc= 0.9000 val_loss= 0.9862 val_acc= 0.7967 time= 0.0275
Epoch: 0092 train_loss= 0.6708 train_acc= 0.9071 val_loss= 0.9814 val_acc= 0.7933 time= 0.0258
Epoch: 0093 train_loss= 0.6627 train_acc= 0.9214 val_loss= 0.9780 val_acc= 0.7967 time= 0.0191
Epoch: 0094 train_loss= 0.6556 train_acc= 0.9214 val_loss= 0.9760 val_acc= 0.7900 time= 0.0242
Epoch: 0095 train_loss= 0.6498 train_acc= 0.9286 val_loss= 0.9736 val_acc= 0.7867 time= 0.0245
Epoch: 0096 train_loss= 0.6442 train_acc= 0.9357 val_loss= 0.9703 val_acc= 0.7833 time= 0.0185
Epoch: 0097 train_loss= 0.6378 train_acc= 0.9357 val_loss= 0.9657 val_acc= 0.7833 time= 0.0191
Epoch: 0098 train_loss= 0.6299 train_acc= 0.9357 v

Epoch: 0178 train_loss= 0.3667 train_acc= 0.9786 val_loss= 0.7581 val_acc= 0.8167 time= 0.0217
Epoch: 0179 train_loss= 0.3658 train_acc= 0.9786 val_loss= 0.7579 val_acc= 0.8167 time= 0.0244
Epoch: 0180 train_loss= 0.3633 train_acc= 0.9786 val_loss= 0.7531 val_acc= 0.8167 time= 0.0241
Epoch: 0181 train_loss= 0.3608 train_acc= 0.9714 val_loss= 0.7487 val_acc= 0.8233 time= 0.0235
Epoch: 0182 train_loss= 0.3587 train_acc= 0.9714 val_loss= 0.7427 val_acc= 0.8433 time= 0.0246
Epoch: 0183 train_loss= 0.3577 train_acc= 0.9714 val_loss= 0.7390 val_acc= 0.8400 time= 0.0200
Epoch: 0184 train_loss= 0.3575 train_acc= 0.9714 val_loss= 0.7384 val_acc= 0.8333 time= 0.0235
Epoch: 0185 train_loss= 0.3561 train_acc= 0.9714 val_loss= 0.7395 val_acc= 0.8333 time= 0.0192
Epoch: 0186 train_loss= 0.3546 train_acc= 0.9714 val_loss= 0.7409 val_acc= 0.8333 time= 0.0224
Epoch: 0187 train_loss= 0.3521 train_acc= 0.9714 val_loss= 0.7424 val_acc= 0.8333 time= 0.0177
Epoch: 0188 train_loss= 0.3504 train_acc= 0.9786 v

In [14]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7723
accuracy = 0.8280


# Gaussian Graph Convolution

In [15]:
B1 = 5e-4
B2 = 5e-4

In [16]:
H = Dropout(0.5)(X_in)
H1 = GaussianGraphConvolution(16,
    is_first=True,
    activation='relu',
    mean_regularizer=l2(B1),
    variance_regularizer=l2(B1),
#     dropout=0.5
)([H]+G)
Y = GaussianGraphConvolution(y.shape[1],
    is_last=True,
    activation='softmax',
)(H1+G)

W1110 18:28:20.746490 139734109144896 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/ggcl.py:174: Normal.__init__ (from tensorflow.python.ops.distributions.normal) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
W1110 18:28:20.747943 139734109144896 deprecation.py:323] From /home/bitcommander/.local/lib/python3.7/site-packages/tensorflow/python/ops/distributions/normal.py:160: Distribution.__init__ (from tensorflow.python.ops.distributions.distribution) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should updat

In [17]:
model = Model(inputs=[X_in]+G, outputs=Y)
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01))
model.add_loss(kl_reg(*H1, B2), H1)

W1110 18:28:20.784544 139734109144896 deprecation.py:323] From /home/bitcommander/Desktop/robust-graph-convolutional-networks-against-adversarial-attacks-implementation/ggcn/losses.py:28: kl_divergence (from tensorflow.python.ops.distributions.kullback_leibler) is deprecated and will be removed after 2019-01-01.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [18]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 1433)         0                                            
__________________________________________________________________________________________________
dropout_3 (Dropout)             (None, 1433)         0           input_2[0][0]                    
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
gaussian_graph_convolution_1 (G [(None, 16), (None,  45856       dropout_3[0][0]                  
                                                                 input_1[0][0]                    
__________

In [19]:
NB_EPOCH = 200
PATIENCE = 10  # early stopping patience
# Helper variables for main training loop
wait = 0
preds = None
best_val_loss = 99999

# Fit
for epoch in range(1, NB_EPOCH+1):

    # Log wall-clock time
    t = time.time()

    # Single training iteration (we mask nodes without labels for loss calculation)
    model.fit(graph, y_train, sample_weight=train_mask,
              batch_size=A.shape[0], epochs=1, shuffle=False, verbose=0)

    # Predict on full dataset
    preds = model.predict(graph, batch_size=A.shape[0])

    # Train / validation scores
    train_val_loss, train_val_acc = evaluate_preds(preds, [y_train, y_val],
                                                   [idx_train, idx_val])
    print("Epoch: {:04d}".format(epoch),
          "train_loss= {:.4f}".format(train_val_loss[0]),
          "train_acc= {:.4f}".format(train_val_acc[0]),
          "val_loss= {:.4f}".format(train_val_loss[1]),
          "val_acc= {:.4f}".format(train_val_acc[1]),
          "time= {:.4f}".format(time.time() - t))

    # Early stopping
    if train_val_loss[1] < best_val_loss:
        best_val_loss = train_val_loss[1]
        wait = 0
    else:
        if wait >= PATIENCE:
            print('Epoch {}: early stopping'.format(epoch))
            break
        wait += 1

Epoch: 0001 train_loss= 1.9431 train_acc= 0.3000 val_loss= 1.9438 val_acc= 0.2567 time= 0.4663
Epoch: 0002 train_loss= 1.9375 train_acc= 0.4143 val_loss= 1.9397 val_acc= 0.3000 time= 0.0238
Epoch: 0003 train_loss= 1.9312 train_acc= 0.4143 val_loss= 1.9335 val_acc= 0.4133 time= 0.0270
Epoch: 0004 train_loss= 1.9245 train_acc= 0.4214 val_loss= 1.9271 val_acc= 0.4400 time= 0.0185
Epoch: 0005 train_loss= 1.9151 train_acc= 0.4786 val_loss= 1.9204 val_acc= 0.4567 time= 0.0229
Epoch: 0006 train_loss= 1.9059 train_acc= 0.4857 val_loss= 1.9132 val_acc= 0.4733 time= 0.0257
Epoch: 0007 train_loss= 1.8968 train_acc= 0.5286 val_loss= 1.9057 val_acc= 0.4700 time= 0.0264
Epoch: 0008 train_loss= 1.8864 train_acc= 0.5429 val_loss= 1.8980 val_acc= 0.4833 time= 0.0248
Epoch: 0009 train_loss= 1.8761 train_acc= 0.5571 val_loss= 1.8906 val_acc= 0.5000 time= 0.0191
Epoch: 0010 train_loss= 1.8654 train_acc= 0.5500 val_loss= 1.8824 val_acc= 0.4833 time= 0.0228
Epoch: 0011 train_loss= 1.8525 train_acc= 0.5786 v

Epoch: 0095 train_loss= 0.6906 train_acc= 0.9143 val_loss= 0.9938 val_acc= 0.7833 time= 0.0227
Epoch: 0096 train_loss= 0.6840 train_acc= 0.9143 val_loss= 0.9892 val_acc= 0.7833 time= 0.0203
Epoch: 0097 train_loss= 0.6778 train_acc= 0.9143 val_loss= 0.9845 val_acc= 0.7833 time= 0.0232
Epoch: 0098 train_loss= 0.6720 train_acc= 0.9143 val_loss= 0.9797 val_acc= 0.7767 time= 0.0231
Epoch: 0099 train_loss= 0.6660 train_acc= 0.9143 val_loss= 0.9753 val_acc= 0.7767 time= 0.0184
Epoch: 0100 train_loss= 0.6606 train_acc= 0.9143 val_loss= 0.9710 val_acc= 0.7767 time= 0.0185
Epoch: 0101 train_loss= 0.6545 train_acc= 0.9143 val_loss= 0.9666 val_acc= 0.7833 time= 0.0186
Epoch: 0102 train_loss= 0.6489 train_acc= 0.9143 val_loss= 0.9626 val_acc= 0.7833 time= 0.0184
Epoch: 0103 train_loss= 0.6433 train_acc= 0.9143 val_loss= 0.9583 val_acc= 0.7833 time= 0.0230
Epoch: 0104 train_loss= 0.6380 train_acc= 0.9143 val_loss= 0.9540 val_acc= 0.7833 time= 0.0185
Epoch: 0105 train_loss= 0.6328 train_acc= 0.9143 v

Epoch: 0191 train_loss= 0.3851 train_acc= 0.9857 val_loss= 0.7704 val_acc= 0.8167 time= 0.0278
Epoch: 0192 train_loss= 0.3837 train_acc= 0.9857 val_loss= 0.7687 val_acc= 0.8133 time= 0.0286
Epoch: 0193 train_loss= 0.3814 train_acc= 0.9857 val_loss= 0.7669 val_acc= 0.8133 time= 0.0190
Epoch: 0194 train_loss= 0.3803 train_acc= 0.9857 val_loss= 0.7653 val_acc= 0.8100 time= 0.0241
Epoch: 0195 train_loss= 0.3783 train_acc= 0.9786 val_loss= 0.7636 val_acc= 0.8100 time= 0.0190
Epoch: 0196 train_loss= 0.3762 train_acc= 0.9857 val_loss= 0.7619 val_acc= 0.8100 time= 0.0188
Epoch: 0197 train_loss= 0.3747 train_acc= 0.9786 val_loss= 0.7605 val_acc= 0.8100 time= 0.0241
Epoch: 0198 train_loss= 0.3729 train_acc= 0.9786 val_loss= 0.7588 val_acc= 0.8100 time= 0.0187
Epoch: 0199 train_loss= 0.3706 train_acc= 0.9786 val_loss= 0.7571 val_acc= 0.8100 time= 0.0195
Epoch: 0200 train_loss= 0.3688 train_acc= 0.9786 val_loss= 0.7558 val_acc= 0.8100 time= 0.0189


In [20]:
test_loss, test_acc = evaluate_preds(preds, [y_test], [idx_test])
print(f"""
loss = {test_loss[0]:.4f}
accuracy = {test_acc[0]:.4f}
""".strip())

loss = 0.7987
accuracy = 0.8210


# Attacks